In [104]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

In [116]:
def compute_distance(x,y):
    if len(y)==0 or len(x)==0:
        print "ERROR: computing distance"
        return 99.9
    m = np.empty([len(y),len(x)])
    for i in range(len(y)):
        # m[i] = np.linalg.norm(A-B[i], 1, axis=1)
        m[i] = np.abs( x - y[i] ).sum(axis=1)
    return np.min(m, axis = 0).sum() / len(x)

In [106]:
def print_percentage(n, t):
    sys.stdout.write('\r')
    sys.stdout.write("[%-20s] %d%%" % ('=' * ((n * 20/t) + 1) , n * 100/t + 1 ))
    sys.stdout.flush()

In [107]:
corpdir = '/fileserver/nmec-handwriting/stil-writing-corpus-processed/seamfeats/FR'
#corpdir = '/fileserver/nmec-handwriting/stil-writing-corpus-processed/vertfeats/FR'
corpdir = '/fileserver/nmec-handwriting/stil-writing-corpus-processed/jaincleanfeats/seamfeats'
corpdir = '/fileserver/nmec-handwriting/stil-writing-corpus-processed/jaincleanfeats/vertfeats'
#corpdir = '/fileserver/nmec-handwriting/stil-writing-corpus-processed/adaptThresh/jainfeats/seamfeats'


#corpdir = '/fileserver/icdar13/jimbo/processed/seamfeats/'
#corpdir = '/fileserver/icdar13/jimbo/processed/TASfeats/'

files = os.listdir(corpdir)
files.sort()

feature_map = []
for i, filename in enumerate(files):
    path = corpdir + "/" + filename
    if os.path.isfile(path):
        feature_file = open(path, 'r').read().splitlines()
        alphabet = np.array([np.fromstring(line, dtype=np.float32, sep=' ')[1:129] for line in feature_file])
        feature_map.append(alphabet)
    print_percentage(i,len(files))
feature_map = np.array(feature_map)

[====================] 100%

In [108]:
feature_map[-1].shape
print len(feature_map)
print len(files)

720
721


In [113]:
compute_distance(feature_map[0], feature_map[1])

0.97112294435501101

In [ ]:
metric = []
for i, image in enumerate(feature_map):
    metricline = [np.array([compute_distance(image, other) for other in feature_map])]
    metric += metricline
    print_percentage(i, len(feature_map))

metric = np.array(metric)
F = -metric
np.fill_diagonal(F, -sys.maxint)

ERROR: computing distance
ERROR: computing distance
[=                   ] 1%ERROR: computing distance
ERROR: computing distance
[=                   ] 1%ERROR: computing distance
ERROR: computing distance
[=                   ] 1%ERROR: computing distance
ERROR: computing distance
[=                   ] 1%ERROR: computing distance
ERROR: computing distance
[=                   ] 1%ERROR: computing distance
ERROR: computing distance
[=                   ] 1%ERROR: computing distance
ERROR: computing distance
[=                   ] 1%ERROR: computing distance
ERROR: computing distance
[=                   ] 1%ERROR: computing distance
ERROR: computing distance
[=                   ] 2%ERROR: computing distance
ERROR: computing distance
[=                   ] 2%ERROR: computing distance
ERROR: computing distance
[=                   ] 2%ERROR: computing distance
ERROR: computing distance
[=                   ] 2%ERROR: computing distance
ERROR: computing distance
[=                   ] 2

In [127]:
F = np.load('CleanJainVert.npy')

soft_correct = 0
hard_correct = 0
total_num = 0

k = 10
g = 8
max_top = 3

for j, i in enumerate(F):
    topk = i.argsort()[-k:]
    if j/g in topk/g:
        soft_correct += 1
    total_num += 1
    hardsample = sum([1 for jj in (j/g == topk[-max_top:]/g) if jj])
    if hardsample == max_top:
        hard_correct += 1
        # print j

print "Top %d = %f" %(k, (soft_correct + 0.0) / total_num)
print "Top %d = %f" %(max_top, (hard_correct + 0.0) / total_num)

Top 10 = 0.783333
Top 3 = 0.404167


In [126]:
soft_correct = 0
hard_correct = 0
total_num = 0

k = 10
g = 8
max_top = 1

for comparej in xrange(8):
    for j, i in enumerate(F):
        if j==comparej:    
            topk = i.argsort()[-k:]
            if j/g in topk/g:
                soft_correct += 1
            total_num += 1
            hardsample = sum([1 for jj in (j/g == topk[-max_top:]/g) if jj])
            if hardsample == max_top:
                hard_correct += 1
    print "Doc %d Top %d = %f" %(comparej, k, (soft_correct + 0.0) / total_num)
    print "Doc %d Top %d = %f" %(comparej, max_top, (hard_correct + 0.0) / total_num)
    


Doc 0 Top 10 = 1.000000
Doc 0 Top 1 = 1.000000
Doc 1 Top 10 = 0.500000
Doc 1 Top 1 = 0.500000
Doc 2 Top 10 = 0.666667
Doc 2 Top 1 = 0.666667
Doc 3 Top 10 = 0.750000
Doc 3 Top 1 = 0.750000
Doc 4 Top 10 = 0.800000
Doc 4 Top 1 = 0.800000
Doc 5 Top 10 = 0.833333
Doc 5 Top 1 = 0.833333
Doc 6 Top 10 = 0.857143
Doc 6 Top 1 = 0.857143
Doc 7 Top 10 = 0.875000
Doc 7 Top 1 = 0.875000


In [123]:
# np.save('AdaptThresh.npy', F)
F=np.load('CleanJainSeam.npy')